In [26]:
import numpy as np
import pandas as pd
from scipy.integrate import cumulative_trapezoid

In [27]:
data_csv = pd.read_csv("hq_markup_train.csv")
bq_data_csv = pd.read_csv("markup_train.csv")

На всякий случай проверка на пустые ряды

In [22]:
blank = 0
blank_t = 0
blank_delta_p = 0
blank_dp = 0
blank_t_and_delta_p = 0
blank_t_and_dp = 0
blank_delta_p_and_dp = 0

for file_name in bq_data_csv['file_name']:
    try:
        txt_file = pd.read_csv(f'data/{file_name}', delimiter='\t', names=['t', 'delta_p', 'dp'])

    except Exception as e:
        print(f"Ошибка при загрузке файла {file_name}: {e}")
        continue

    if len(txt_file) == 0:
        blank += 1
        continue

    txt_file = pd.DataFrame(txt_file)

    if txt_file.shape[1] < 3:
        print(f"Файл {file_name} содержит менее трех столбцов.")
        continue

    # Проверка на пустые столбцы
    is_t_empty = txt_file['t'].isna().all() or (txt_file['t'] == 0).all()
    is_delta_p_empty = txt_file['delta_p'].isna().all() or (txt_file['delta_p'] == 0).all()
    is_dp_empty = txt_file['dp'].isna().all() or (txt_file['dp'] == 0).all()

    # Подсчет пустых столбцов
    if is_t_empty:
        blank_t += 1
    if is_delta_p_empty:
        blank_delta_p += 1
    if is_dp_empty:
        blank_dp += 1

    # Подсчет комбинаций пустых столбцов
    if is_t_empty and is_delta_p_empty:
        blank_t_and_delta_p += 1
    if is_t_empty and is_dp_empty:
        blank_t_and_dp += 1
    if is_delta_p_empty and is_dp_empty:
        blank_delta_p_and_dp += 1

print(f"Пустых файлов: {blank}")
print(f"Пустых столбцов t: {blank_t}")
print(f"Пустых столбцов delta_p: {blank_delta_p}")
print(f"Пустых столбцов dp: {blank_dp}")
print(f"Пустых t и delta_p: {blank_t_and_delta_p}")
print(f"Пустых t и dp: {blank_t_and_dp}")
print(f"Пустых delta_p и dp: {blank_delta_p_and_dp}")

Пустых файлов: 257
Пустых столбцов t: 0
Пустых столбцов delta_p: 0
Пустых столбцов dp: 0
Пустых t и delta_p: 0
Пустых t и dp: 0
Пустых delta_p и dp: 0


In [29]:
indices_to_drop = []
print(bq_data_csv.shape)

for index, file_name in enumerate(bq_data_csv['file_name']):
    txt_file = pd.read_csv(f'data/{file_name}', delimiter='\t', names=['t', 'delta_p', 'dp'])
    if len(txt_file) == 0:
        indices_to_drop.append(index)

bq_data_csv = bq_data_csv.drop(indices_to_drop).reset_index(drop=True)
bq_data_csv.shape

(45141, 18)


(44884, 18)

In [30]:
def smooth(y, window_size):
    """Сглаживание данных методом скользящего среднего"""
    if window_size <= 1:
        return y
    window = np.ones(window_size) / window_size
    return np.convolve(y, window, mode='same')

Хз, правильно ли работает окно, код от чатгпт

In [33]:
def process_file(file_path):
    data = pd.read_csv(f'data/{file_name}', delimiter='\t', names=['t', 'deltaP', 'dP'])
    if data.size == 0:
        return None  # Пропускаем пустые файлы
    
    t = data['t']
    delta_p = data['deltaP']
    dp = data['dP']

    # Вычисление размера окна для сглаживания
    n_points = len(t) if t is not None else len(data)
    window_size = int(0.5 * np.log10(n_points))  # 0.5 лог-цикла
    window_size = max(1, window_size)  # Минимум 1 точка

    # Сглаживание имеющихся данных
    if delta_p is not None:
        delta_p = smooth(delta_p, window_size)
    if dp is not None:
        dp = smooth(dp, window_size)
    
    # Сборка результата
    processed_data = []
    if t is not None:
        processed_data.append(t)
    if delta_p is not None:
        processed_data.append(delta_p)
    if dp is not None:
        processed_data.append(dp)
    
    return np.column_stack(processed_data) if processed_data else None

In [34]:
for file_name in bq_data_csv['file_name']:
    input_path = f"data/{file_name}"
    output_path = f"processed_data/{file_name}"
    
    result = process_file(input_path)
    if result is not None:
        np.savetxt(output_path, result)
    else:
        print(f"Ошибка обработки файла: {file_name}")